In [ ]:
import json
import numpy as np
import cv2

import os
import pickle
from typing import Tuple, List, Dict

In [ ]:
PATH = 'resources'
CLASSES = {'un-classified': '4', 'destroyed': '3', 'major-damage': '2', 'minor-damage': '1', 'no-damage': '0'}

def check_directories(path: str) -> Tuple[str]:
    """Valida la integridad de los directorios.

    Args:
        path (str): ruta raíz a los directorios a validar.

    Returns:
        Tuple[str]: las tres rutas del conjunto de entrenamiento, prueba y validación.
    """
    if not os.path.isdir(path):
        raise Exception(f'there are not a valid path')

    train_path = os.path.join(path, 'train')
    if not os.path.isdir(train_path):
        raise Exception(f'You must add the train folder {train_path}')
    
    test_path= os.path.join(path, 'test')
    if not os.path.isdir(test_path):
        raise Exception(f'You must add the train folder {test_path}')
    
    val_path= os.path.join(path, 'hold')
    if not os.path.isdir(val_path):
        raise Exception(f'You must add the train folder {val_path}')
    
    return train_path, test_path, val_path


def json_to_yolo(_path: str, dst: str) -> Dict[str, List[np.ndarray]]:
    """Crea para cada imagen un archivo de etiqueta que el 
    modelo YOLO pueda entender:

    <clase> <coordenadas del poligono>

    Args:
        _path (str): ruta a las etiquetas originales.
        dst (str): ruta de destino a las etiquetas en formato YOLO.

    Returns:
        Dict[str, List[np.ndarray]]: contiene el par de 
        nombre del archivo contra la lista de los poligonos.
    """
    if not os.path.exists(dst):
        os.makedirs(dst)

    polygons_per_image = {}
    for file in os.listdir(_path):

        file_name, ext = os.path.splitext(file)
        if 'pre' in file_name:
            continue
        if ext == '.json':
            print(f'proceesing file {file_name}')
            with open(os.path.join(_path, file)) as f:
                j = json.load(f)

            polygons = []
            final = ""
            combined_file_name = file_name.replace('post', 'combined')
            new_txt = os.path.join(dst, f'{combined_file_name}.txt')
            
            with open(new_txt, 'w') as t:
                for _e in j['features']['xy']:
                    st_p = CLASSES[_e['properties']["subtype"]]
                    pairs = [coords for coords in _e['wkt'][10:-2].split(", ")]
                    nd_p = ""
                    coordinates = []
                    for pair in pairs:
                        x, y = pair.split(" ")
                        x = round(float(x), 4)
                        y = round(float(y), 4)
                        coordinates.append([x,y])
                        nd_p += " " + str(x/1024) + " " + str(y/1024)
                    final = st_p + nd_p
                    polygons.append(np.array(coordinates, dtype=int))
                    print(final)
                    t.write(final+'\n')
            print(f'file converted to YOLO format: {combined_file_name}.txt')
            polygons_per_image[combined_file_name] = polygons

    return polygons_per_image
            
def create_input_images(_path: str, dst: str):
    """Como entrada del modelo YOLO se crea una imagen superpuesta.
    Canal R: imagen antes del desastre.
    Canal G: imagen después del desastre.
    Canal B: Diferencia entre ambas imagenes.

    Args:
        _path (str): ruta a las imagenes originales.
        dst (str): ruta de destino a las imagenes en formato super puesto.
    """
    if not os.path.exists(dst):
        os.makedirs(dst)
    for pre_file in os.listdir(_path):
        if 'post' in pre_file:
            continue

        print(f'proccessing image: {pre_file}')
        post_file = pre_file.replace('pre', 'post')
        img = cv2.imread(os.path.join(_path, pre_file))
        img2 = cv2.imread(os.path.join(_path, post_file))
        g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        g_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        in_ = np.stack((g_img, g_img2, g_img2-g_img), axis=2)

        combined_image_name = pre_file.replace('pre', 'combined')
        combined_path = os.path.join(dst, combined_image_name)
        
        print(f'merged into: {combined_path}')
        cv2.imwrite(combined_path, in_)


In [ ]:
def do_format(path: str):
    """Da el formato necesario al dataset para la entrada del modelo.

    Args:
        path (str): ruta raiz del dataset del modelo
    """
    paths = check_directories(path)
    
    steps = ('train', 'test', 'val')

    _data = os.path.join(path, 'data')
    _images = os.path.join(_data, 'images')
    _labels = os.path.join(_data, 'labels')

    for path_, step in zip(paths, steps):
        print(f'processing path: {path_} for step {step}')
        original_images = os.path.join(path_, 'images')
        create_input_images(original_images, os.path.join(_images, step))

        original_labels = os.path.join(path_, 'labels')
        label_dict = json_to_yolo(original_labels, os.path.join(_labels, step))
        with open(os.path.join(PATH, f'{step}.pickle'), 'wb') as f:
            pickle.dump(label_dict, f)

    
    

In [12]:
do_format(PATH)

processing path: resources/train for step train
proccessing image: hurricane-harvey_00000015_pre_disaster.png
merged into: resources/data/images/train/hurricane-harvey_00000015_combined_disaster.png
proccessing image: hurricane-harvey_00000228_pre_disaster.png
merged into: resources/data/images/train/hurricane-harvey_00000228_combined_disaster.png
proccessing image: hurricane-michael_00000261_pre_disaster.png
merged into: resources/data/images/train/hurricane-michael_00000261_combined_disaster.png
proccessing image: hurricane-harvey_00000216_pre_disaster.png
merged into: resources/data/images/train/hurricane-harvey_00000216_combined_disaster.png
proccessing image: hurricane-harvey_00000413_pre_disaster.png
merged into: resources/data/images/train/hurricane-harvey_00000413_combined_disaster.png
proccessing image: hurricane-florence_00000102_pre_disaster.png
merged into: resources/data/images/train/hurricane-florence_00000102_combined_disaster.png
proccessing image: socal-fire_00001040_p

In [ ]:
def cut_dataset(_path: str, percentage: float = 1/3):
    """Recorta el dataset a cierta proporción.

    Args:
        _path (str): ruta del dataset a recortar.
        percentage (float, optional): Defaults to 1/3.
    """
    for step in ('train', 'test', 'val'):
        step_path = os.path.join(_path, step)

        files = sorted(os.listdir(step_path))
        threshold = len(files)*percentage
        print(f'threshold for {step}: {threshold}')
        for c, file in enumerate(files):
            if c > threshold:
                file_to_remove = os.path.join(step_path, file)
                print(f'removing: {file_to_remove}')
                os.remove(file_to_remove) 

cut_dataset('resources/data/images')
cut_dataset('resources/data/labels')

threshold for train: 933.0
removing: resources/data/images/train/hurricane-michael_00000059_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000060_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000061_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000066_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000067_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000068_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000072_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000075_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000076_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000077_combined_disaster.png
removing: resources/data/images/train/hurricane-michael_00000078_combined_disaster.png
removing: resour

In [17]:

with open('resources/train/labels/guatemala-volcano_00000000_post_disaster.json') as f:
    d = json.load(f)
    print(d)

{'features': {'lng_lat': [{'properties': {'feature_type': 'building', 'subtype': 'no-damage', 'uid': '486b0813-ecd2-4b84-856c-9c0e42156953'}, 'wkt': 'POLYGON ((-90.81544679490855 14.39086318334812, -90.81537467350067 14.39060467857134, -90.81584174451893 14.39043032647906, -90.81586635209965 14.39049581582557, -90.81593344431286 14.39048145754227, -90.81595559689623 14.39057367091926, -90.81587964155047 14.39059650626524, -90.81590706308843 14.39071123556855, -90.81544679490855 14.39086318334812))'}, {'properties': {'feature_type': 'building', 'subtype': 'no-damage', 'uid': '139cf2c8-ad52-4739-82b5-bb646b215e76'}, 'wkt': 'POLYGON ((-90.81420592027385 14.38829423654861, -90.81421178336095 14.38832937106001, -90.81408837763853 14.38834314934208, -90.81404547543569 14.38816089961379, -90.81410016818667 14.38815172795384, -90.8141122317199 14.38818732301324, -90.81427959937533 14.38816185446788, -90.81428289441583 14.38820583612065, -90.81412306222495 14.38822210022881, -90.8141326064717 1

In [4]:
d['features']['xy']

[{'properties': {'feature_type': 'building',
   'subtype': 'major-damage',
   'uid': '104d384e-412b-4a72-8361-3c6c75cafdf0'},
  'wkt': 'POLYGON ((212.2425241572 0.005244624768526229, 231.3821676979141 12.88017282398865, 217.5595676779494 33.35326908503746, 209.474650679756 26.3115671854467, 203.73696765205 33.87487663350699, 175.048552510671 11.96735961487916, 180.2646279882456 7.012087908216943, 173.1979822446708 0.00499755545006233, 212.2425241572 0.005244624768526229))'},
 {'properties': {'feature_type': 'building',
   'subtype': 'major-damage',
   'uid': '9badd81e-16bf-4406-b8ad-9cb50a208011'},
  'wkt': 'POLYGON ((111.0102139006811 45.00256966117878, 137.9000536406594 26.43577555933709, 159.6680191395227 52.84543958614845, 129.0968322972 74.13322937754333, 111.4903896105185 51.40491245711114, 114.3714438690686 49.48420961918605, 111.3303310417597 44.84251109313215, 111.0102139006811 45.00256966117878))'},
 {'properties': {'feature_type': 'building',
   'subtype': 'major-damage',
  

In [ ]:
polygons = []
final = ""
#with open('prueba.txt', 'w') as f:
for _e in d['features']['xy']:
    st_p = CLASSES[_e['properties']['subtype']]
    pairs = [coords for coords in _e['wkt'][10:-2].split(", ")]
    nd_p = ""
    coordinates = []
    for pair in pairs:
        x, y = pair.split(" ")
        x = round(float(x), 4)
        y = round(float(y), 4)
        coordinates.append([x,y])
        nd_p += " " + str(x/1024) + " " + str(y/1024)
    final = st_p + nd_p
    polygons.append(np.array(coordinates, dtype=int))
    print(final)
polygons
        #f.write(final)
    #coordinates.append([[float(c) for c in coords.split(" ")] for coords in _e['wkt'][10:-2].split(", ")])

0 0.51971328125 0.16209228515625 0.535744140625 0.21890751953125 0.4357869140625 0.25827744140625 0.43036474609375 0.24389677734375 0.41598408203125 0.247197265625 0.4110333984375 0.22692294921875 0.4273 0.2217365234375 0.4211705078125 0.19651142578125 0.51971328125 0.16209228515625
0 0.79160771484375 0.725638671875 0.790275 0.71790869140625 0.8167525390625 0.71462119140625 0.8263484375 0.75469296875 0.8146201171875 0.756825390625 0.81195458984375 0.74900654296875 0.7760587890625 0.75495947265625 0.7752591796875 0.7452748046875 0.8095556640625 0.74136533203125 0.807334375 0.72332861328125 0.80537958984375 0.72395048828125 0.8052908203125 0.725638671875 0.80102587890625 0.72626064453125 0.8004927734375 0.72510556640625 0.79160771484375 0.725638671875
0 0.99950869140625 0.83462880859375 0.9802513671875 0.838354296875 0.97865205078125 0.83711044921875 0.96692373046875 0.83808779296875 0.96692373046875 0.83719931640625 0.95874951171875 0.837376953125 0.95404033203125 0.80894462890625 0.999

[array([[532, 165],
        [548, 224],
        [446, 264],
        [440, 249],
        [425, 253],
        [420, 232],
        [437, 227],
        [431, 201],
        [532, 165]]),
 array([[810, 743],
        [809, 735],
        [836, 731],
        [846, 772],
        [834, 774],
        [831, 766],
        [794, 773],
        [793, 763],
        [828, 759],
        [826, 740],
        [824, 741],
        [824, 743],
        [820, 743],
        [819, 742],
        [810, 743]]),
 array([[1023,  854],
        [1003,  858],
        [1002,  857],
        [ 990,  858],
        [ 990,  857],
        [ 981,  857],
        [ 976,  828],
        [1023,  822],
        [1023,  854]]),
 array([[1023,  937],
        [1014,  939],
        [1003,  901],
        [1020,  897],
        [1023,  909],
        [1023,  937]]),
 array([[1023,  988],
        [ 997,  999],
        [ 983,  968],
        [ 971,  972],
        [ 968,  958],
        [ 980,  955],
        [ 964,  907],
        [ 999,  894],
      

In [ ]:
from pathlib import Path

yaml_content = f'''
path: /home/ec2-user/SageMaker/dataset-yolo
train: train
val: validation
test: test

# Classes - use the class_map as guide
names:
  0: person
  1: car
'''

with Path(os.path.join(target_path, 'seg_dataset.yaml')).open('w') as f:
    f.write(yaml_content)

In [22]:
for c in coordinates:
    print(c)

[(212.2425241572, 0.005244624768526229), (231.3821676979141, 12.88017282398865), (217.5595676779494, 33.35326908503746), (209.474650679756, 26.3115671854467), (203.73696765205, 33.87487663350699), (175.048552510671, 11.96735961487916), (180.2646279882456, 7.012087908216943), (173.1979822446708, 0.00499755545006233), (212.2425241572, 0.005244624768526229)]
[(111.0102139006811, 45.00256966117878), (137.9000536406594, 26.43577555933709), (159.6680191395227, 52.84543958614845), (129.0968322972, 74.13322937754333), (111.4903896105185, 51.40491245711114), (114.3714438690686, 49.48420961918605), (111.3303310417597, 44.84251109313215), (111.0102139006811, 45.00256966117878)]
[(70.80872744088899, 136.8631511105073), (76.89898531916401, 129.5941336450958), (96.34851853210614, 147.0790675456624), (73.75562641300729, 173.4046983625819), (71.00518737231607, 170.8507192514191), (68.05828840067255, 173.6011582923471), (85.54322230480402, 188.9250329469789), (86.52552195790368, 188.3356531529353), (91

In [26]:
coordinates= coordinates.astype(int)
coordinates

array([[ 119, 1022],
       [ 112, 1012],
       [ 146,  990],
       [ 165, 1022],
       [ 119, 1022]])

In [52]:
img = cv2.imread('resources/train/images/guatemala-volcano_00000000_post_disaster.png')
#n_img = cv2.polylines(img, pts=polygons, isClosed=False, color=(0,255,0), thickness=4, lineType=16)
cv2.imshow('prueba', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [53]:
img2 = cv2.imread('resources/train/images/guatemala-volcano_00000000_pre_disaster.png')

In [ ]:
img = cv2.imread('resources/train/images/guatemala-volcano_00000000_post_disaster.png')
img2 = cv2.imread('resources/train/images/guatemala-volcano_00000000_pre_disaster.png')
g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
g_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
in_ = np.stack((g_img, g_img2, g_img2-g_img), axis=2)

In [50]:
(g_img2-g_img).shape

(1024, 1024)

In [62]:
in_ = np.stack((g_img, g_img2, g_img2-g_img), axis=2)


In [ ]:
n_in_ = cv2.polylines(in_, pts=polygons, isClosed=False, color=(0,255,0), thickness=4, lineType=16)
cv2.imshow('prueba', n_in_)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
#cv2.imshow('prueba', g_img2-g_img)
n_in_ = cv2.polylines(g_img, pts=polygons, isClosed=False, color=(0,255,0), thickness=4, lineType=16)
cv2.imshow('prueba', n_in_)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
import numpy as np
np.array(coordinates)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (20,) + inhomogeneous part.

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11l-seg.pt")

# Define the path to the image file
source = "path/to/image.jpg"


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/XMK0406/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11l-seg.pt... <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>


######################################################################## 100.0%#########################                                         45.9%###########################################                            64.6%



0: 640x640 (no detections), 378.4ms
Speed: 7.7ms preprocess, 378.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


In [63]:

# Run inference on the source
results = model(in_) 

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen


0: 640x640 1 tv, 1 cell phone, 421.2ms
Speed: 7.2ms preprocess, 421.2ms inference, 11.0ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
model.train(data='config.yaml', epochs=1, imgsz= 1040)

New https://pypi.org/project/ultralytics/8.3.230 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.11.3 torch-2.9.1 CPU (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=config.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1040, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, 

Matplotlib is building the font cache; this may take a moment.


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics